In [2]:
import pandas as pd

data = pd.read_csv(r"C:\Users\kosiv\OneDrive\Desktop\BBB\M259\archive\synthetic_beverage_sales_data.csv", sep=';')
print(data.head())
data.info()



  Order_ID Customer_ID Customer_Type             Product     Category  \
0     ORD1     CUS1496           B2B          Vio Wasser        Water   
1     ORD1     CUS1496           B2B               Evian        Water   
2     ORD1     CUS1496           B2B              Sprite  Soft Drinks   
3     ORD1     CUS1496           B2B  Rauch Multivitamin       Juices   
4     ORD1     CUS1496           B2B        Gerolsteiner        Water   

   Unit_Price  Quantity  Discount  Total_Price             Region  Order_Date  
0        1.66        53      0.10        79.18  Baden-Württemberg  23.08.2023  
1        1.56        90      0.10       126.36  Baden-Württemberg  23.08.2023  
2        1.17        73      0.05        81.14  Baden-Württemberg  23.08.2023  
3        3.22        59      0.10       170.98  Baden-Württemberg  23.08.2023  
4        0.87        35      0.10        27.40  Baden-Württemberg  23.08.2023  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
D

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Wähle Features (X) und Zielvariable (y)
X = data[['Customer_Type', 'Category', 'Unit_Price', 'Quantity', 'Discount', 'Total_Price']]
y = data['Region']

# One-hot Encoding für kategorische Felder
X_encoded = pd.get_dummies(X, drop_first=True)

# Datensatz aufteilen
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Modell trainieren
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Wichtige Features bestimmen
importances = model.feature_importances_
features = X_encoded.columns
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': importances}).sort_values(by='Importance', ascending=False)

print(feature_importance_df.head(10))


                Feature  Importance
3           Total_Price    0.430072
0            Unit_Price    0.369684
1              Quantity    0.188593
2              Discount    0.007574
6  Category_Soft Drinks    0.001548
5       Category_Juices    0.001330
7        Category_Water    0.001004
4     Customer_Type_B2C    0.000193


In [8]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Parameter-Raum definieren
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Random Forest vorbereiten
rf = RandomForestClassifier(random_state=42)

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Suche starten
random_search.fit(X_train, y_train)

# Bestes Modell
best_model = random_search.best_estimator_

# Neue Vorhersagen
y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Optimierte Modellgenauigkeit: {accuracy_best:.2%}")



Fitting 3 folds for each of 20 candidates, totalling 60 fits
Optimierte Modellgenauigkeit: 22.55%


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Parameterraum
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Randomized Search Setup
rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Modelltraining
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_


Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [9]:
from sklearn.metrics import accuracy_score

# Vorhersagen mit dem getunten Modell
y_pred_best = best_model.predict(X_test)


In [10]:
cm = confusion_matrix(y_test, y_pred_best, labels=best_model.classes_)
print("Confusion Matrix:")
print(cm)

print("\nRecall-Werte (Sensitivität) pro Klasse:")
report = classification_report(y_test, y_pred_best, output_dict=True)
for label, metrics in report.items():
    if label not in ['accuracy', 'macro avg', 'weighted avg']:
        print(f"{label}: Recall = {metrics['recall']:.2f}")




Confusion Matrix:
[[2593 3634 3021 1771 2192]
 [1920 4620 3261 1944 2318]
 [1830 3920 3542 1901 2411]
 [1873 3931 3155 2133 2484]
 [1770 3838 3451 1842 2444]]

Recall-Werte (Sensitivität) pro Klasse:
Bremen: Recall = 0.20
Hamburg: Recall = 0.33
Niedersachsen: Recall = 0.26
Rheinland-Pfalz: Recall = 0.16
Sachsen: Recall = 0.18


## Teil 4.4 – Zusammenfassung & Hypothesen

Das Modell erreicht nach Hyperparameter-Tuning auf die fünf häufigsten Regionen eine Genauigkeit von **22.55 %**. Dies stellt eine deutliche Verbesserung gegenüber der ursprünglichen Genauigkeit von 7.18 % dar.

**Recall-Werte (Sensitivität):**
- Bremen: 20 %
- Hamburg: 33 %
- Niedersachsen: 26 %
- Rheinland-Pfalz: 16 %
- Sachsen: 18 %

**Interpretation:**
Die Region **Hamburg** wird vom Modell am besten erkannt. Dies kann an klareren Kaufmustern oder einer größeren Repräsentation im Datensatz liegen. Andere Regionen wie **Rheinland-Pfalz** oder **Sachsen** zeigen niedrigere Recall-Werte, was auf Überschneidungen im Kaufverhalten mit anderen Regionen hindeutet.

**Hypothesen zur Modellleistung:**
- Preis- und Mengendaten allein reichen nur begrenzt zur Regionenvorhersage aus.
- Die besten Prädiktoren waren `Total_Price`, `Unit_Price` und `Quantity`.
- Weitere Verbesserungen wären mit geografischen, zeitlichen oder benutzerspezifischen Merkmalen möglich.
